<a href="https://colab.research.google.com/github/FreddyPinto/recsys-steam-games/blob/feature/notebooks/3.1-functions-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Functions Test


El objetivo de este notebook es crear las funciones para los endpoints que se consumirán en la API. Además, se realizarán pruebas para asegurar su funcionamiento adecuado utilizando los dataset creados específicamente para este fin.


## 0 Configuraciones Globales e Importaciones


En esta sección, importamos todas las bibliotecas y/o modulos necesarios para nuestro proceso de feature engineering y establecemos configuraciones globales de ser requerido.


In [2]:
import sys
import pandas as pd

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")

System version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Pandas version: 2.1.3


## 1 Extracción


En esta sección, extraemos los datos de los archivos `pseudo_db1` y `pseudo_db2` en formato parquet que serán utilizados como pseudo base de datos para los endpoints.


### 1.1 Extracción de los datos


Creamos una función que lee cada archivo desde su directorio y lo carga a un DataFrame de `pandas`.


In [3]:
# Cargamos los archivos parquet
def read_parquet_files(parquet_files):
    dataframes = {}
    for name in parquet_files:
        dataframes[name] = pd.read_parquet(
            f'../data/processed/{name}.parquet', engine='pyarrow')
    return dataframes


parquet_files = ['pseudo-db1', 'pseudo-db2', 'user_sim', 'items_sim', 'matrix']
dataframes = read_parquet_files(parquet_files)

# Convertimos a df.
df_pseudo_db1 = dataframes['pseudo-db1']
df_pseudo_db2 = dataframes['pseudo-db2']
df_user_sim = dataframes['user_sim']
df_item_sim = dataframes['items_sim']
df_matrix = dataframes['matrix']

## 2 Endpoints


### 2.1 Endpoint 1


def **PlayTimeGenre( _`genero` : str_ )**:
Retorna `año` con mas horas jugadas para el género dado.
Ejemplo de retorno:

```js
{
   "Año de lanzamiento con más horas jugadas para Género X": 2013
}
```


In [76]:
def PlayTimeGenre(genre: str):
    """
    Devuelve el año con más horas jugadas para un género dado.

    Args:
        genre: El género del juego.

    Returns:
        El año de lanzamiento con más horas jugadas para un género dado.
    Ejemplo de retorno:
        {
            "Año de lanzamiento con más horas jugadas para Género X": 2013
        }
    """

    # Verifica si el argumento es un str
    if not isinstance(genre, str):
        # Devuelve un mensaje de error
        return f"El género debe ser un string. Por favor, ingrese un género válido."

    # Convierte el género a título o a mayúsculas según corresponda
    if genre.upper() == "RPG":
        genre = genre.upper()
    else:
        genre = genre.title()

    # Verifica si el género existe
    if genre not in df_pseudo_db1.columns:
        # Devuelve un mensaje de error
        return f"Genre {genre} not found"

    # Filtra por género y resetea el índice para poder obtener el año de lanzamiento correcto
    df_genre = df_pseudo_db1[genre].reset_index()

    # Agrupa por año y calcula la suma total de horas jugadas para cada año
    playtime_by_year = df_genre.groupby('release_year')[genre].sum()

    # Encuentra el año con más horas jugadas
    max_playtime_year = (playtime_by_year.idxmax())
    response = {
        f"Año de lanzamiento con más horas jugadas para el género {genre}": max_playtime_year}

    return response

In [77]:
PlayTimeGenre('Casual')

{'Año de lanzamiento con más horas jugadas para el género Casual': 2015}

In [81]:
PlayTimeGenre('ActiOn')

{'Año de lanzamiento con más horas jugadas para el género Action': 2012}

In [78]:
PlayTimeGenre('CaSuals')

'Genre Casuals not found'

In [82]:
PlayTimeGenre(12)

'El género debe ser un string. Por favor, ingrese un género válido.'

### 2.2 Endpoint 2


- def **UserForGenre( _`genero` : str_ )**:
  Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno:

```js
{
   "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
   "Horas jugadas":[
      {
         "Año":2013,
         "Horas":203
      },
      {
         "Año":2012,
         "Horas":100
      },
      {
         "Año":2011,
         "Horas":23
      }
   ]
}
```


In [57]:
def UserForGenre(genre: str):
  """
  Devuelve el usuario que acumula más horas jugadas para un género dado.

  Args:
    genre: El género del juego.

  Returns:
    El usuario con más horas jugadas y una lista de la acumulación de horas jugadas por año.
    Ejemplo de retorno:
        {
          "Usuario con más horas jugadas para Género X":"us213ndjss09sdf",
          "Horas jugadas":[
              {
                "Año":2013,
                "Horas":203
              },
              {
                "Año":2012,
                "Horas":100
              },
              {
                "Año":2011,
                "Horas":23
              }
          ]
        }
    """
  # Verificar si el argumento es un str
  if not isinstance(genre, str):
      # Devolver un mensaje de error
      return f"El argumento debe ser un str. Por favor, ingrese un género válido."
  
  # Convertir el argumento a título o a mayúsculas según corresponda
  if genre.upper() =="RPG":
      genre = genre.upper()
  else:
      genre = genre.title()

  if genre not in df_pseudo_db1.columns:
        # Devolver un mensaje de error
        return f"Genre {genre} not found"
  
  df_genre = df_pseudo_db1[genre].reset_index()
  user_max_hours = df_genre.groupby('user_id')[genre].sum().idxmax()
  hours_by_year = df_genre[df_genre['user_id'] == user_max_hours].groupby('release_year')[genre].sum().reset_index()
  hours_by_year.columns = ['Año', 'Horas']
  hours_by_year = hours_by_year[hours_by_year['Horas'] > 0]
  hours_by_year['Horas'] = hours_by_year['Horas'].round().astype(int)

  response = {
      f"Usuario con más horas jugadas para Género {genre} " : user_max_hours,
      "Horas jugadas": hours_by_year.to_dict('records')
  }

  return response


In [62]:
UserForGenre('rPg')

{'Usuario con más horas jugadas para Género RPG ': 'shinomegami',
 'Horas jugadas': [{'Año': 1999, 'Horas': 6},
  {'Año': 2000, 'Horas': 140},
  {'Año': 2003, 'Horas': 8849},
  {'Año': 2004, 'Horas': 30},
  {'Año': 2005, 'Horas': 3},
  {'Año': 2006, 'Horas': 34},
  {'Año': 2007, 'Horas': 2278},
  {'Año': 2008, 'Horas': 22},
  {'Año': 2009, 'Horas': 23},
  {'Año': 2010, 'Horas': 690},
  {'Año': 2011, 'Horas': 351},
  {'Año': 2012, 'Horas': 459},
  {'Año': 2013, 'Horas': 2108},
  {'Año': 2014, 'Horas': 2456},
  {'Año': 2015, 'Horas': 2132},
  {'Año': 2016, 'Horas': 259}]}

In [59]:
UserForGenre('actiOn')

{'Usuario con más horas jugadas para Género Action ': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 4},
  {'Año': 1997, 'Horas': 4},
  {'Año': 1999, 'Horas': 1},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2001, 'Horas': 4},
  {'Año': 2002, 'Horas': 4},
  {'Año': 2003, 'Horas': 129},
  {'Año': 2004, 'Horas': 2124},
  {'Año': 2005, 'Horas': 356},
  {'Año': 2006, 'Horas': 1504},
  {'Año': 2007, 'Horas': 1880},
  {'Año': 2008, 'Horas': 142},
  {'Año': 2009, 'Horas': 1813},
  {'Año': 2010, 'Horas': 1379},
  {'Año': 2011, 'Horas': 2591},
  {'Año': 2012, 'Horas': 6404},
  {'Año': 2013, 'Horas': 2036},
  {'Año': 2014, 'Horas': 2214},
  {'Año': 2015, 'Horas': 6572},
  {'Año': 2016, 'Horas': 503},
  {'Año': 2017, 'Horas': 722}]}

In [60]:
UserForGenre('Ajjioni')

'El género Ajjioni no está en el dataframe. Por favor, ingrese un género válido.'

In [61]:
UserForGenre(123)

{'El argumento debe ser un str. Por favor, ingrese un género válido.'}

### 2.3 Endpoint 3


- def **UsersRecommend( _`año` : int_ )**:
  Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno:

```js
[
  {
    "Puesto 1": "X",
  },
  {
    "Puesto 2": "Y",
  },
  {
    "Puesto 3": "Z",
  },
];
```


In [97]:
def UsersRecommend(year: int):
    """
    Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos/neutrales)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de juegos recomendados.
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_pseudo_db2['posted_year'].isin([year]).any():
        print(f"Year {year} not found")
        return None

    # Filtra el DataFrame por año y recomendación
    df_year = df_pseudo_db2[(df_pseudo_db2['posted_year'] == year) & (
        df_pseudo_db2['recommend'] == True) & (df_pseudo_db2['sentiment_analysis'] > 1)]

    # Agrupa por juego y cuenta el número de recomendaciones para cada juego
    recommendations = df_year.groupby('item_name').size()

    # Ordena los juegos en función del número de recomendaciones y toma el top 3
    top_games = recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game}
                for i, game in enumerate(top_games.index)]

    return response

In [105]:
UsersRecommend(2021)

Year 2021 not found


In [99]:
UsersRecommend(2010)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Killing Floor'},
 {'Puesto 3': 'Alien Swarm'}]

### 2.4 Endpoint 4


- def **UsersWorstDeveloper( _`año` : int_ )**:
  Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno:

```js
[
  {
    "Puesto 1": "X",
  },
  {
    "Puesto 2": "Y",
  },
  {
    "Puesto 3": "Z",
  },
];
```


In [106]:
def UsersWorstDeveloper(year: int):
    """
    Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

    Args:
      year: El año a filtrar.

    Returns:
      El top 3 de desarrolladoras con juegos MENOS recomendados
      Ejemplo de retorno:
          [
            {
                "Puesto 1":"X"
            },
            {
                "Puesto 2":"Y"
            },
            {
                "Puesto 3":"Z"
            }
          ]
    """
    # Comprueba si el año se encuentra en el DataFrame
    if ~df_pseudo_db2['posted_year'].isin([year]).any():
        print(f"Year {year} not found")
        return None

    # Filtra el DataFrame por año y no recomendación
    df_year = df_pseudo_db2[(df_pseudo_db2['posted_year'] == year) & (
        df_pseudo_db2['recommend'] == False) & (df_pseudo_db2['sentiment_analysis'] == 0)]

    # Agrupa por juego y cuenta el número de no recomendaciones para cada juego
    not_recommendations = df_year.groupby('developer').size()

    # Ordena los juegos en función del número de no recomendaciones y toma el top 3
    top_games = not_recommendations.sort_values(ascending=False).head(3)

    response = [{"Puesto {}".format(i+1): game}
                for i, game in enumerate(top_games.index)]

    return response

In [107]:
UsersWorstDeveloper(2025)

Year 2025 not found


In [11]:
UsersWorstDeveloper(2014)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Bohemia Interactive'},
 {'Puesto 3': 'Facepunch Studios'}]

### 2.5 Endpoint 5


def **sentiment_analysis( _`empresa desarrolladora` : str_ )**:
Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total
de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno:

```js
{
   "Valve":[
      {
         "Negative":1352
      },
      {
         "Neutral":2202
      },
      {
         "Positive":4840
      }
   ]
}
```


In [67]:
def sentiment_analysis(developer: str):
    """
    Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

    Args:
      developer: La desarrolladora a filtrar.

    Returns:
      El análisis de sentimiento para la desarrolladora dada.
      Ejemplo de retorno:
          {
            "Valve":[
                {
                  "Negative":1352
                },
                {
                  "Neutral":2202
                },
                {
                  "Positive":4840
                }
            ]
          }
    """
    # Convierte el nombre de la desarrolladora a minúsculas
    developer_lower = developer.lower()

    # Filtra el DataFrame por el nombre de la empresa desarrolladora
    df_dev = df_pseudo_db2[df_pseudo_db2["developer"].str.lower()
                           == developer_lower]

    # Verifica si el desarrollador ingresado existe en el DataFrame
    if df_dev.empty:
        print(f"Developer {developer} not found")
        return None

    # Contamos el número de registros por cada categoría de sentimiento
    sentiment_counts = df_dev['sentiment_analysis'].value_counts()
    negative_reviews = sentiment_counts.get(0, 0)
    neutral_reviews = sentiment_counts.get(1, 0)
    positive_reviews = sentiment_counts.get(2, 0)

    # Crear el diccionario con el nombre de la empresa desarrolladora como llave y la lista con los conteos como valor
    response = {
        developer_lower.capitalize(): [
            {"Negative": negative_reviews},
            {"Neutral": neutral_reviews},
            {"Positive": positive_reviews}
        ]
    }

    return response

In [68]:
sentiment_analysis('Valve')

{'Valve': [{'Negative': 1352}, {'Neutral': 2202}, {'Positive': 4840}]}

In [69]:
sentiment_analysis('VAlVe')

{'Valve': [{'Negative': 1352}, {'Neutral': 2202}, {'Positive': 4840}]}

In [71]:
sentiment_analysis('V54aDSA')

Developer V54aDSA not found


## 3 Recommendations

### 3.1 Item-Item

+ def **game_recommender( *`item_name`* )**:
    Ingresando el nombre de un juego, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [5]:
def game_recommender(item_name: str):
    '''
    Muestra una lista de juegos similares a un juego dado.

    Args:
        item_name: El nombre del juego para el cual se desean encontrar juegos similares.

    Returns:
        Los 5 juegos recomendados.

    '''
    # Verificamos si el juego ingresado está en el dataframe
    if item_name not in df_item_sim.index:
      return f"Game {item_name} not found"

    # Buscamos la fila del dataframe que corresponde al juego ingresado
    row = df_item_sim.loc[item_name]

    # Ordenamos la fila de mayor a menor similitud
    row_sorted = row.sort_values(ascending=False)

    # Obtenemos los nombres de los 5 juegos más similares, excluyendo el mismo juego
    similar_games = row_sorted.index[1:6]
    
    # Creamos un diccionario vacío para guardar los juegos y sus índices
    recommendations = {}

    for i, juego in enumerate(similar_games, start=1):
      # Asignamos el índice y el nombre del juego al diccionario
      recommendations[i] = juego
    return recommendations

In [6]:
game_recommender('Reign ')

'Game Reign  not found'

### 3.2 User-Item

+ def **recomendacion_usuario( *`user_id`* )**:
    Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.